In [1]:
from __future__ import division
import time
from os import listdir
from os.path import isfile, join
import torch, os
import torch.nn as nn
import random
import math
import copy, time, pickle, datetime
import pandas as pd
import numpy as np
from numpy import linalg as LA

In [2]:
# set file path and parameter

In [3]:
lamb = random.choice([0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000])  # lambda initialization
val_batch_size = 0.9  # data points for validation
K = 100  # few shots in support
Kq = 2 * K  # shots for query
inner_steps = 1  # gradient steps in the inner loop
pd_updates = 1  # inner primal-dual iteration
num_iterations = 100 # outer iteration
meta_batch = 4

eta_1 = 0.001  # step size of inner primal update
eta_2 = 0.001  # step size of inner dual update
eta_3 = 0.001  # step size of outer primal update
eta_4 = 0.001  # step size of outer dual update
xi = 0.015  # parameter of outer regularization
delta = 50  # some constant for outer augmentation
eps = 0.05  # fairness threshold
num_neighbors = 3

# cls_syn_data:2; adult:16; communities_and_crime:100; bank:16; census_income:36
d_feature = 16  # feature size of the data set
data_path = r'/home/mifeng/FFML-main/raw_data'
dataset = r'adult'
save = r'your_save'
tasks = [x[0] for x in os.walk(data_path + '/' + dataset)][1:]
#     print("tasks",tasks)
start = time.time()
print(lamb)



0.1


In [4]:
def running_time(totalsec):
    day = totalsec // (24 * 3600)
    restsec = totalsec % (24 * 3600)
    hour = restsec // 3600
    restsec %= 3600
    minutes = restsec // 60
    restsec %= 60
    seconds = restsec
    print("Total running time: %d days, %d hours, %d minutes, %d seconds." % (day, hour, minutes, seconds))

In [5]:
def meta_update(d_feature, net, lamb,
                K, Kq, num_neighbors,
                num_iterations, buffer, inner_steps, pd_updates, meta_batch,
                eta_1, eta_2, eta_3, eta_4, delta, eps, xi):
    weights = list(net.parameters())

    try:
        lamb = torch.tensor([copy.deepcopy(lamb)], requires_grad=True, dtype=torch.float)
    except:
        lamb = lamb.clone()

    if len(buffer) <= meta_batch:
        batch = copy.deepcopy(buffer)
    else:
        batch = random.sample(buffer, meta_batch)

    for iter in range(1, num_iterations + 1):
        meta_loss = 0
        for i in range(len(batch)):
            task = batch[i]
            t_loss, t_fair, t_acc, t_dp, t_eop, t_disc, t_cons = cal_loss_and_fairness(d_feature, net, lamb, task,
                                                                                       K, Kq, num_neighbors,
                                                                                       inner_steps, pd_updates,
                                                                                       eta_1, eta_2, eps)
            if type(t_fair) is not str:
                meta_loss += (t_loss + lamb * t_fair - (delta * eta_4 / 2) * (lamb ** 2))
            else:
                meta_loss += (t_loss - (delta / 2) * (lamb ** 2))

        reg = (net.e_norm(weights)) ** 2
        meta_loss = meta_loss + xi * reg

        meta_grads = torch.autograd.grad(meta_loss, weights, retain_graph=True)
        temp_weights = [w.clone() for w in weights]
        weights = [w - eta_3 * g for w, g in zip(temp_weights, meta_grads)]
        weights_norm = net.e_norm(weights)
        if weights_norm > 1:
            weights = list(nn.parameter.Parameter(item / weights_norm) for item in weights)
        else:
            weights = list(nn.parameter.Parameter(item) for item in weights)

        net.assign(weights)

        grad_lamb = torch.autograd.grad(meta_loss, lamb)
        lamb = lamb + eta_4 * grad_lamb[0]
        if lamb.item() < 0:
            lamb = torch.tensor([0], requires_grad=True, dtype=torch.float)

    return net, lamb


def prep(save, d_feature, lamb, dataset,
         K, Kq, val_batch_size, num_neighbors,
         num_iterations, inner_steps, pd_updates, meta_batch,
         eta_1, eta_2, eta_3, eta_4, delta, eps, xi):
    now = datetime.datetime.now()
    exp_name = now.strftime("\%Y-%m-%d-%H-%M-%S-Ours-" + dataset)
    save_folder = save + exp_name
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    # model_save_path = save_folder + r"\model.pth"
    val_save_path = save_folder + r"/val.txt"
    with open(save_folder + r'/hyper-parameters.txt', 'wb') as f:
        t = hyper_params(d_feature, lamb, dataset,
                         K, Kq, val_batch_size, num_neighbors,
                         num_iterations, inner_steps, pd_updates, meta_batch,
                         eta_1, eta_2, eta_3, eta_4, delta, eps, xi)
        f.write(t)

    return val_save_path


def pdrftml(d_feature, lamb, tasks, data_path, dataset, save,
            K, Kq, val_batch_size, num_neighbors,
            num_iterations, inner_steps, pd_updates, meta_batch,
            eta_1, eta_2, eta_3, eta_4, delta, eps, xi):
    val_save_path = prep(save, d_feature, lamb, dataset,
                         K, Kq, val_batch_size, num_neighbors,
                         num_iterations, inner_steps, pd_updates, meta_batch,
                         eta_1, eta_2, eta_3, eta_4, delta, eps, xi)
    net = NN(d_feature)
    lamb = copy.deepcopy(lamb)
    buffer = []
    T = len(tasks)
    res = []
#     print("enter pdrftml")
#     print("T",T)
    for t in range(1, T + 1):
        start_time = time.time()
        # print("task" + str(t))
        task0 = pd.read_csv(data_path + '/' + dataset + r'/task' + str(t) + r'/task' + str(t) + '_neg.csv')
        task1 = pd.read_csv(data_path + '/' + dataset + r'/task' + str(t) + r'/task' + str(t) + '_pos.csv')
        task = [task0, task1]
#         print("task",task)
        buffer.append(task)

        # evaluation
        ###########################################################################################################################################################
        loss_val, fair_val, accuracy_val, dp_val, eop_val, discrimination_val, consistency_val = cal_loss_and_fairness(d_feature, net, lamb, task,
                                                                                                                       K, val_batch_size, num_neighbors,
                                                                                                                       inner_steps, pd_updates,
                                                                                                                       eta_1, eta_2, eps)
        cost_time = time.time() - start_time
        print("Val-Task %s/%s: loss:%s; dbc:%s; acc:%s ;dp:%s; eop:%s; disc:%s; cons:%s; time:%s sec." % (
            t, T, np.round(loss_val.item(), 4), np.round(fair_val, 10), np.round(accuracy_val, 10), np.round(dp_val, 10), np.round(eop_val, 10),
            np.round(discrimination_val, 10), np.round(consistency_val, 10),
            np.round(cost_time, 4)))

        # torch.save(net.state_dict(), model_save_path)
        res.append([loss_val.item(), fair_val, accuracy_val, dp_val, eop_val, discrimination_val, consistency_val, cost_time])
        ###########################################################################################################################################################

        # meta-train
        new_net, new_lamb = meta_update(d_feature, net, lamb,
                                        K, Kq, num_neighbors,
                                        num_iterations, buffer, inner_steps, pd_updates, meta_batch,
                                        eta_1, eta_2, eta_3, eta_4, delta, eps, xi)

        net = new_net
        lamb = new_lamb

    with open(val_save_path, 'wb') as f:
        pickle.dump(res, f)


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NN(nn.Module):
    def __init__(self, d_in):
        super(NN, self).__init__()
        self.linear1 = nn.Linear(d_in, 40)
        self.linear2 = nn.Linear(40, 40)
        self.linear3 = nn.Linear(40, 1)

    def forward(self, x):
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        x = F.relu(x)
        x = self.linear3(x)
        return torch.sigmoid(x)

    def parameterised(self, x, weights):
        # like forward, but uses ``weights`` instead of ``model.parameters()``
        # it'd be nice if this could be generated automatically for any nn.Module...
        x = nn.functional.linear(x, weights[0], weights[1])
        x = nn.functional.relu(x)
        x = nn.functional.linear(x, weights[2], weights[3])
        x = nn.functional.relu(x)
        x = nn.functional.linear(x, weights[4], weights[5])
        return torch.sigmoid(x)

    def assign(self, weights):
        with torch.no_grad():
            self.linear1.weight = weights[0]
            self.linear1.bias = weights[1]
            self.linear2.weight = weights[2]
            self.linear2.bias = weights[3]
            self.linear3.weight = weights[4]
            self.linear3.bias = weights[5]

    def e_norm(self, weights):
        out = 0
        for weight in weights:
            out += torch.norm(weight)
        return out

In [7]:
from texttable import Texttable


def hyper_params(d_feature, lamb, dataset,
                 K, Kq, val_batch_size, num_neighbors,
                 num_iterations, inner_steps, pd_updates, meta_batch,
                 eta_1, eta_2, eta_3, eta_4, delta, eps, xi):
    table = Texttable()
    table.set_deco(Texttable.HEADER)
    table.set_cols_align(["l", "l"])
    table.add_rows([
        ["Parameters", "Values"],
        ["Data", dataset],
        ["d_feature", d_feature],
        ["Lambda", lamb],
        ["Support shots", K],
        ["Query shots", Kq],
        ["Validation shots", val_batch_size],
        ["Number of neighbors", num_neighbors],
        ["Inner gradient steps", inner_steps],
        ["Primal-Dual Iterates", pd_updates],
        ["Outer loops", num_iterations],
        ["Meta batches", meta_batch],
        ["eta_1", eta_1],
        ["eta_2", eta_2],
        ["eta_3", eta_3],
        ["eta_4", eta_4],
        ["delta", delta],
        ["eps", eps],
        ["xi", xi]
    ])
    return table.draw().encode()


In [8]:
from __future__ import division
import torch, os
import torch.nn as nn
import random
import math
import copy, time, pickle, datetime
import pandas as pd
import numpy as np
from numpy import linalg as LA
from sklearn.metrics import accuracy_score



def sample_data_from_task(task, batch_size, d_feature):
    data = task.sample(batch_size)
    X = data[data.columns[-d_feature:]].copy()
    y = data[["y"]]
    z = data[["z"]]
    return X, y, z


def mean(a):
    return sum(a).to(dtype=torch.float) / len(a)


def cal_loss_and_fairness(d_feature, net, lamb, task,
                          K, Kq, num_neighbors,
                          inner_steps, pd_updates,
                          eta_1, eta_2, eps):
    temp_weights = [w.clone() for w in list(net.parameters())]
    try:
        temp_lambda = torch.tensor([copy.deepcopy(lamb)], requires_grad=True, dtype=torch.float)
    except:
        temp_lambda = lamb.clone()

    criterion = nn.BCELoss()

    task0 = task[0]
    task1 = task[1]
    task_df = pd.concat([task0, task1])

    X0_s, y0_s, z0_s = sample_data_from_task(task0, K, d_feature)
    X1_s, y1_s, z1_s = sample_data_from_task(task1, K, d_feature)
    X_s = pd.concat([X0_s, X1_s]).values
    y_s = pd.concat([y0_s, y1_s]).values
    z_s = pd.concat([z0_s, z1_s]).values
    z_bar = np.mean(z_s) * np.ones((len(z_s), 1))

    X_s = torch.tensor(X_s, dtype=torch.float).unsqueeze(1)
    y_s = torch.tensor(y_s, dtype=torch.float).unsqueeze(1)
    ones = torch.tensor(np.ones((len(y_s), 1)), dtype=torch.float).unsqueeze(1)
    z_s = torch.tensor(z_s, dtype=torch.float).unsqueeze(1)
    z_bar = torch.tensor(z_bar, dtype=torch.float).unsqueeze(1)

    for co_update in range(pd_updates):
        for step in range(inner_steps):
            y_hat = net.parameterised(X_s, temp_weights)
            fair = torch.abs(torch.mean((z_s - z_bar) * y_hat)) - eps
            loss = (-1.0) * torch.mean(y_s * torch.log(y_hat) + (ones - y_s) * torch.log(ones - y_hat)) + temp_lambda * fair
            loss = loss / K
            grad = torch.autograd.grad(loss.sum(), temp_weights, retain_graph=True)
            temp_weights = [w - eta_1 * g for w, g in zip(temp_weights, grad)]
            new_y_hat = net.parameterised(X_s, temp_weights)
            fair = torch.abs(torch.mean((z_s - z_bar) * new_y_hat)) - eps
            loss = (-1.0) * torch.mean(y_s * torch.log(new_y_hat) + (ones - y_s) * torch.log(ones - new_y_hat)) + temp_lambda * fair
            loss = loss / K
            grad_lamb = torch.autograd.grad(loss.sum(), temp_lambda)
            temp_lambda = temp_lambda + eta_2 * grad_lamb[0]
            if temp_lambda.item() < 0:
                temp_lambda = torch.tensor([0], requires_grad=True, dtype=torch.float)

    if Kq < 1:
        Kq = round(len(task_df.index) * Kq)

    X_q, y_q, z_q = sample_data_from_task(task_df, Kq, d_feature)
    X_q = X_q.values
    y_q = y_q.values
    z_q = z_q.values

    X_temp = copy.deepcopy(X_q)
    z_temp = copy.deepcopy(z_q)
    y_temp = copy.deepcopy(y_q)
    z_bar = np.mean(z_q) * np.ones((len(z_q), 1))

    X_q = torch.tensor(X_q, dtype=torch.float).unsqueeze(1)
    y_q = torch.tensor(y_q, dtype=torch.float).unsqueeze(1)
    z_q = torch.tensor(z_q, dtype=torch.float).unsqueeze(1)
    z_bar = torch.tensor(z_bar, dtype=torch.float).unsqueeze(1)

    y_hat = net.parameterised(X_q, temp_weights)
    loss = criterion(y_hat, y_q)
    loss = loss / Kq

    fair = torch.abs(torch.mean((z_q - z_bar) * y_hat)).item()

    y_hat = y_hat.detach().numpy().reshape(len(y_hat), 1)
    y_q = y_q.detach().numpy().reshape(len(y_q), 1)

    input_zy = np.column_stack((z_temp, y_hat))
    z_y_hat_y = np.column_stack((input_zy, y_temp))
    yX = np.column_stack((y_hat, X_temp))

    accuracy = accuracy_score(y_hat.round(), y_q)
    dp = cal_dp(input_zy)
    eop = cal_eop(z_y_hat_y)
    discrimination = cal_discrimination(input_zy)
    # consistency = cal_consistency(yX, num_neighbors)
    consistency = 1

    return loss, fair, accuracy, dp, eop, discrimination, consistency


In [9]:
import torch
import math
import numpy as np


def cal_discrimination(input_zy):
    a_values = []
    b_values = []
    for line in input_zy:
        if line[0] == 0:
            a_values.append(line[1])
        elif line[0] == 1:
            b_values.append(line[1])

    if len(a_values) == 0:
        discrimination = sum(b_values) * 1.0 / len(b_values)
    elif len(b_values) == 0:
        discrimination = sum(a_values) * 1.0 / len(a_values)
    else:
        discrimination = sum(a_values) * 1.0 / len(a_values) - sum(b_values) * 1.0 / len(b_values)
    return abs(discrimination)


# calculate the Euclidean distance between two vectors
def euclidean_distance(row1, row2):
    distance = 0.0
    for i in range(len(row1) - 1):
        distance += (row1[i] - row2[i]) ** 2
    return math.sqrt(distance)


# Locate the most similar neighbors
# example: get_neighbors(yX, X[0], 3)
def get_neighbors(yX, target_row, num_neighbors):
    distances = list()
    for yX_row in yX:
        X_row = yX_row[1:]
        y = yX_row[0]
        dist = euclidean_distance(target_row, X_row)
        distances.append((y, dist))
    distances.sort(key=lambda tup: tup[1])
    neighbors = list()
    for i in range(num_neighbors):
        neighbors.append(distances[i][0])
    return neighbors


def cal_consistency(yX, num_neighbors):
    ans = 0
    for yX_row in yX:
        temp = 0
        target_row = yX_row[1:]
        target_y = yX_row[0]
        y_neighbors = get_neighbors(yX, target_row, num_neighbors)
        for y_neighbor in y_neighbors:
            temp += abs(target_y - y_neighbor)
        ans += temp
    return (1 - (ans * 1.0) / (len(yX) * num_neighbors))


def cal_dbc(input_zy):
    length = len(input_zy)
    z_bar = np.mean(input_zy[:, 0])
    dbc = 0
    for zy in input_zy:
        dbc += (zy[0] - z_bar) * zy[1] * 1.0
    return abs(dbc / length)

def cal_dp(input_zy):
    count1 = 0
    count2 = 0
    for item in input_zy:
        if item[0] == 0:
            count1 += 1
            if item[1].round() == 1:
                count2 += 1
    try:
        dp = abs(1 - count2 * 1.0 / count1)
    except:
        dp = 0
    return dp

def cal_eop(z_y_hat_y):
    count1 = 0
    count2 = 0
    for item in z_y_hat_y:
        if item[0] == 0 and item[2] == 1:
            count1 += 1
            if item[1].round() == 1:
                count2 += 1
    try:
        eop = abs(1 - count2 * 1.0 / count1)
    except:
        eop = 0
    return eop

In [ ]:
pdrftml(d_feature, lamb, tasks, data_path, dataset, save,K, Kq, val_batch_size, num_neighbors,num_iterations, inner_steps, pd_updates, meta_batch,eta_1, eta_2, eta_3, eta_4, delta, eps, xi)
cost_time_in_second = time.time() - start
running_time(cost_time_in_second)

Val-Task 1/50: loss:0.0002; dbc:2.00175e-05; acc:0.6976320583 ;dp:0.7384169884; eop:0.6688311688; disc:8.51376e-05; cons:1; time:0.0955 sec.
Val-Task 2/50: loss:0.0003; dbc:5.1e-09; acc:0.8888009497 ;dp:1.0; eop:1.0; disc:0.0; cons:1; time:0.0513 sec.
Val-Task 3/50: loss:0.0003; dbc:2.34e-08; acc:0.7016274864 ;dp:1.0; eop:1.0; disc:0.0; cons:1; time:0.056 sec.
Val-Task 4/50: loss:0.0003; dbc:9.5e-09; acc:0.6741622575 ;dp:1.0; eop:1.0; disc:0.0; cons:1; time:0.0525 sec.
Val-Task 5/50: loss:0.0003; dbc:1.02e-08; acc:0.6995884774 ;dp:1.0; eop:1.0; disc:0.0; cons:1; time:0.0528 sec.
Val-Task 6/50: loss:0.0002; dbc:3.1e-09; acc:0.8830044998 ;dp:1.0; eop:1.0; disc:0.0; cons:1; time:0.0565 sec.
Val-Task 7/50: loss:0.0002; dbc:1.76e-08; acc:0.9148226735 ;dp:1.0; eop:1.0; disc:0.0; cons:1; time:0.0638 sec.
Val-Task 8/50: loss:0.0003; dbc:1.29e-08; acc:0.9002061856 ;dp:1.0; eop:1.0; disc:0.0; cons:1; time:0.0931 sec.
Val-Task 9/50: loss:0.0003; dbc:1.48e-08; acc:0.8855550793 ;dp:1.0; eop:1.0; di